In [2]:
# check gpu
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# load data and split into train and test sets
from load_data import sim_arr
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(sim_arr, test_size=0.2, random_state=42)

In [ ]:
from autoencoder import Autoencoder
from hyper_optim import Hyper

# hyperparameter optimization
hyper = Hyper(Autoencoder, 3, train_set, test_set)
study = hyper.optimize(direction="minimize",
                       study_name="autoencoder",
                       storage="autoencoder.db",
                       n_trials=1)

# Get the best hyperparameters
best_params = study.best_params
for key, value in best_params.items():
    if key == 'activation':
        print(key, ":", value.__name__)
    else:
        print(key, ":", value)
print("Best hyperparameters:", best_params)
print("Best value:", study.best_value)

In [ ]:
# train the model with the best hyperparameters
hyperparams = Hyper.search_space(study.best_trial)
model = Autoencoder(*hyperparams, output_dim=3)

model.fit(train_set, test_set, epochs=10, batch_size=32)
model.evaluate(test_set, test_set)

In [ ]:
# summarize the model
model.get_layer('encoder').summary()
model.get_layer('decoder').summary()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

i = np.random.randint(0, len(test_set))
in_sim = test_set[i: i + 1]
out_sim = model(in_sim)
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(in_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")
ax[1].imshow(out_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")

In [ ]:
from tensorflow.keras.layers import Flatten as Flatten

feature_arr = model.feature_extractor(sim_arr)

In [ ]:
plt.scatter(feature_arr[:, 0],
            feature_arr[:, 1],
            feature_arr[:, 2])